In [1]:
from ultralytics import YOLO
from PIL import Image
from matplotlib import pyplot as plt
import math
import numpy as np
import cv2
from scipy.ndimage import gaussian_filter
from time import time
import socket
import base64
import io
# Load a model
vehicle_model = YOLO("yolov8x.pt")
beacon_model = YOLO("runs\\detect\\train6\\weights\\best.pt")
sock2 = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
UDP_IP = "192.168.5.3" #raspberry pi ip ethernet
UDP_PORT = 12345

In [ ]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock2.sendto(b'start', (UDP_IP, UDP_PORT))
sock.bind(('192.168.5.2', 8000))
sock.listen(1)
print("listening")
conn, addr = sock.accept()

def recvall(sock, count):
    buf = b''
    while count:
        newbuf = sock.recv(count)
        if not newbuf: return None
        buf += newbuf
        count -= len(newbuf)
    return buf

active_beacon_count = 0
currentframe = 0
frame_count = 0
beacon_type_count = [0,0,0,0,0]
long_beacon_area = [0,0,0,0,0,0,0,0,0,0,0,0,0]
while True:
    try:
        sock2.sendto(b'start', (UDP_IP, UDP_PORT))
        length = recvall(conn, 64)
        length1 = length.decode('utf-8')
        stringData = recvall(conn, int(length1))
        f = io.BytesIO(base64.b64decode(stringData))
        img = Image.open(f)
        
        display_image = np.asarray(img)
        vehicle_loc_list = []
        
        vehicle_results = vehicle_model(img, verbose=False, classes=[7,2])
        v_results_array = vehicle_results[0].boxes
        vehicle_list = []
        for i in v_results_array:
            current_box = i.boxes[0]
            class_tensor = current_box[5]
            x1 = int(current_box[0])
            y1 = int(current_box[1]) - 70
            x2 = int(current_box[2])
            y2 = int(current_box[3])
            img_result = img.crop((x1,y1,x2,y2))
            vehicle_list.append(img_result)
            
            #for display
            vehicle_loc_list.append([x1,y1])
            vehicle_bouding_box = cv2.rectangle(display_image, (x1,y1), (x2,y2), (0,255,0), 3)
            vehicle_text_bg = cv2.rectangle(display_image, (x1,y1-30), (x1+120, y1), (0,0,255), -1)
            vehicle_txt = cv2.putText(display_image, 'Vehicle', (x1,y1-3), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

        if vehicle_list:
            beacon_list = []
            classes = []
            for i,v in enumerate(vehicle_list):
                beacon_result = beacon_model(v, verbose=False)

                b_result_array = beacon_result[0].boxes
                smallest_width = 1000
                smallest_width_pos = 0
                long_beacon_count = 0
                firetruck_bwidth = []
                firetruck_bheight = []
                for a,b in enumerate(b_result_array):
                    current_box = b.boxes[0]
                    class_tensor = current_box[5]
                    x1 = int(current_box[0])
                    y1 = int(current_box[1])
                    x2 = int(current_box[2])
                    y2 = int(current_box[3])
                    
                    # for display
                    dis_x1 = vehicle_loc_list[i][0] + x1
                    dis_y1 = vehicle_loc_list[i][1] + y1
                    dis_x2 = vehicle_loc_list[i][0] + x2
                    dis_y2 = vehicle_loc_list[i][1] + y2
                    beacon_bouding_box = cv2.rectangle(display_image, (dis_x1,dis_y1), (dis_x2,dis_y2), (255,0,0), 3)
                    beacon_text_bg = cv2.rectangle(display_image, (dis_x1,dis_y1-30), (dis_x1+120, dis_y1), (255,0,0), -1)
                    beacon_txt = cv2.putText(display_image, 'Beacon', (dis_x1,dis_y1-3), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                    
                    if class_tensor == 4.:
                        bheight = y2-y1
                        bwidth = x2-x1
                        if bheight > 5 and bwidth > 50 and bwidth < 500 and (bwidth/bheight) > 4.78:
                            long_beacon_count += 1
                            if (x2-x1) < smallest_width:
                                smallest_width = x2-x1
                                smallest_width_pos = len(beacon_list)-1
                            if bheight < 15:
                                y2 += 15-bheight
                            img_result = v.crop((x1,y1,x2,y2))
                            beacon_list.append(img_result)
                            classes.append(class_tensor)
                    elif class_tensor == 2.:
                        x1 -= 2
                        y1 -= 1
                        x2 += 2
                        y2 += 5
                        firetruck_bwidth.append([x2,x1])
                        firetruck_bheight.append([y2,y1])
                        img_result = v.crop((x1,y1,x2,y2))
                        beacon_list.append(img_result)
                        classes.append(class_tensor)
                        if a == 1:
                            break
                    else:
                        img_result = v.crop((x1,y1,x2,y2))
                        beacon_list.append(img_result)
                        classes.append(class_tensor)
                if long_beacon_count > 1:
                    del beacon_list[smallest_width_pos]
                    del classes[smallest_width_pos]
                elif len(firetruck_bwidth) > 1:
                    which_bwidth = 0
                    which_bheight = 0
                    width_dif = (firetruck_bwidth[0][0]-firetruck_bwidth[0][1])-(firetruck_bwidth[1][0]-firetruck_bwidth[1][1])
                    height_dif = (firetruck_bheight[0][0]-firetruck_bheight[0][1])-(firetruck_bheight[1][0]-firetruck_bheight[1][1])
                    left_changes = abs(int(width_dif*3/4))
                    right_changes = abs(width_dif) - left_changes
                    bottom_changes = abs(height_dif)
                    if width_dif < 0:
                        which_bwidth = 1
                    if height_dif < 0:
                        which_bheight = 1
                    if left_changes != 0 or right_changes != 0:
                        new_x1 = firetruck_bwidth[which_bwidth][1]+left_changes
                        new_x2 = firetruck_bwidth[which_bwidth][0]-right_changes
                        same_y1 = firetruck_bheight[which_bwidth][1]
                        same_y2 = firetruck_bheight[which_bwidth][0]
                        beacon_list[len(beacon_list)-1-(1-which_bwidth)] = v.crop((new_x1,same_y1,new_x2,same_y2))
                    if bottom_changes != 0:
                        same_x1 = firetruck_bwidth[which_bheight][1]+left_changes
                        same_x2 = firetruck_bwidth[which_bheight][0]-right_changes
                        same_y1 = firetruck_bheight[which_bheight][1]
                        new_y2 = firetruck_bheight[which_bheight][0]-bottom_changes
                        beacon_list[len(beacon_list)-1-(1-which_bheight)] = v.crop((same_x1,same_y1,same_x2,new_y2))
                        
            if beacon_list:
                mid_contour_count = 0
                print("Beacon frame: " + str(currentframe) + " with b num of " + str(len(beacon_list)))
                for bi, b in enumerate(beacon_list):
                    beacon_type_count[int(classes[bi])] += 1

                    height_half_point = int(b.height/2)
                    width_half_point = int(b.width/2)
                    image_height = b.height
                    image_width = b.width
                    quarter_x = int(image_width/4)
                    quarter34_x = image_width-quarter_x
                    one_third = int(image_width/3)
                    two_third = image_width-one_third

                    blurred_image = gaussian_filter(cv2.cvtColor(np.asarray(b),cv2.COLOR_BGR2GRAY),sigma=3)
                    thresh = int(cv2.minMaxLoc(blurred_image)[1] * 0.7)
                    if classes[bi] == 1. or classes[bi] == 2.:
                        thresh = int(cv2.minMaxLoc(blurred_image)[1] * 0.9)
                    elif classes[bi] == 0.:
                        thresh = int(cv2.minMaxLoc(blurred_image)[1] * 0.8)
                        
                    ret, thresh_img = cv2.threshold(blurred_image,thresh, 255, cv2.THRESH_BINARY)
                    contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

                    for cnt in contours:
                        reach_half_x = False
                        reach_half_y = False
                        reach_edge_x = False
                        reach_edge_y = False
                        reach_edge_y_bottom = False
                        reach_quarter_x = False
                        reach_quarter34_x = False
                        reach_quarter_x_half_y = False
                        reach_quarter34_x_half_y = False
                        reach_near_edge_x_count = 0
                        reach_middle_third_count = 0
                        lowest_y = 5000
                        highest_y = 0
                        lowest_x = 5000
                        highest_x = 0
                        firetruck_lowest_loc = [0,5000] #x,y
                        firetruck_highest_loc = [0,0] #x,y
                        for i in cnt:
                            if classes[bi] == 4.:
                                if i[0][1] > highest_y:
                                    highest_y = i[0][1]
                                if i[0][1] < lowest_y:
                                    lowest_y = i[0][1]
                                if i[0][0] >= one_third and i[0][0] <= two_third:
                                    reach_middle_third_count += 1
                                else:
                                    reach_middle_third_count -= 1
                                if i[0][1] >= height_half_point and i[0][1] < height_half_point+5:
                                    reach_half_y = True
                                if i[0][0] >= width_half_point-1 and i[0][0] <= width_half_point+1:
                                    reach_half_x = True
                                if i[0][0] >= quarter_x-1 and i[0][0] <= quarter_x+1:
                                    reach_quarter_x = True
                                    if i[0][1] >= height_half_point and i[0][1] < image_height:
                                        reach_quarter_x_half_y = True
                                if i[0][0] >= quarter34_x-1 and i[0][0] <= quarter34_x+1:
                                    reach_quarter34_x = True
                                    if i[0][1] >= height_half_point and i[0][1] < image_height:
                                        reach_quarter34_x_half_y = True
                                if i[0][0] < quarter_x-1 or i[0][0] > quarter34_x+1:
                                    reach_near_edge_x_count += 1
                                else:
                                    reach_near_edge_x_count -= 1
                                if (i[0][1] >= 0 and i[0][1] < 3) or (i[0][1] > image_height-3 and i[0][1] <= image_height):
                                    reach_edge_y = True
                                    if i[0][1] > image_height-5 and i[0][1] <= image_height:
                                        reach_edge_y_bottom = True
                                if (i[0][0] >= 0 and i[0][0] < 5) or (i[0][0] > image_width-5 and i[0][0] <= image_width):
                                    reach_edge_x = True
                            elif classes[bi] == 2. or classes[bi] == 0.:
                                if i[0][1] > highest_y:
                                    highest_y = i[0][1]
                                if i[0][1] < lowest_y:
                                    lowest_y = i[0][1]
                                if i[0][0] > highest_x:
                                    highest_x = i[0][0]
                                if i[0][0] < lowest_x:
                                    lowest_x = i[0][0]
                                if i[0][1] <= 3 or i[0][1] >= image_height-3:
                                    reach_edge_y = True
                                if i[0][0] <= 3 or i[0][0] >= image_width-3:
                                    reach_edge_x = True
                                if i[0][0] >= width_half_point-1 and i[0][0] <= width_half_point+1:
                                    reach_half_x = True
                                if i[0][1] >= height_half_point-1 and i[0][1] < height_half_point+1:
                                    reach_half_y = True
                                if i[0][1] >= height_half_point-1 and i[0][1] < height_half_point+1:
                                    reach_half_y = True
                            else:
                                if i[0][1] >= height_half_point-2 and i[0][1] < height_half_point+2:
                                    reach_half_y = True
                                if i[0][1] < 5 or i[0][1] > image_height-5 or i[0][0] < 5 or i[0][0] > image_width-5:
                                    reach_edge_y = True
                                if i[0][0] >= width_half_point-1 and i[0][0] <= width_half_point+1:
                                    reach_half_x = True
                                if (i[0][0] >= quarter_x-1 and i[0][0] <= quarter_x+1) or (i[0][0] >= quarter34_x-1 and i[0][0] <= quarter34_x+1):
                                    reach_quarter_x = True 
                        if classes[bi] == 4.:
                            if ((reach_quarter_x_half_y and not reach_quarter34_x) or (reach_quarter34_x_half_y and not reach_quarter_x)) and reach_half_x and not reach_edge_x and not reach_edge_y_bottom and (highest_y-lowest_y)/height_half_point > 0.9: 
                                mid_contour_count += 1
                                long_beacon_area[0] += 1
                            if reach_half_y and (reach_quarter_x or reach_quarter34_x) and not reach_half_x and not (reach_edge_x or reach_edge_y) and (highest_y-lowest_y)/height_half_point > 0.9:
                                mid_contour_count += 1
                                long_beacon_area[1] += 1
                            elif reach_quarter_x_half_y and reach_quarter34_x_half_y and reach_half_x and not reach_edge_x and not reach_edge_y_bottom and (highest_y-lowest_y)/height_half_point >= 1.0:
                                mid_contour_count += 1
                                long_beacon_area[2] += 1
                            elif reach_half_y and reach_half_x and (reach_quarter_x or reach_quarter34_x) and not (reach_edge_x or reach_edge_y) and (highest_y-lowest_y)/height_half_point > 0.9:
                                mid_contour_count += 1
                                long_beacon_area[3] += 1
                            elif reach_half_y and (reach_quarter_x or reach_quarter34_x) and not reach_half_x and not reach_edge_x and not reach_edge_y_bottom and (highest_y-lowest_y)/height_half_point > 0.9:
                                mid_contour_count += 1
                                long_beacon_area[4] += 1
                            elif reach_half_y and (reach_middle_third_count<-10) and not reach_half_x and not(reach_edge_x or reach_edge_y) and reach_near_edge_x_count < -10:
                                mid_contour_count += 1
                                long_beacon_area[5] += 1
                            elif reach_half_y and (reach_middle_third_count < 0) and not reach_half_x and not(reach_edge_x or reach_edge_y) and (highest_y-lowest_y)/height_half_point > 0.7:
                                mid_contour_count += 1
                                long_beacon_area[6] += 1
                            elif (reach_middle_third_count < 0) and not reach_half_x and not(reach_edge_x or reach_edge_y) and image_height <= 20:
                                mid_contour_count += 1
                                long_beacon_area[7] += 1
                            elif reach_half_y and not reach_half_x and not(reach_edge_x or reach_edge_y) and image_height <= 20:
                                mid_contour_count += 1
                                long_beacon_area[8] += 1
                            elif (reach_quarter_x_half_y or reach_quarter34_x_half_y) and not reach_half_y and not reach_edge_y and (highest_y-lowest_y)/height_half_point > 1.0:
                                mid_contour_count += 1
                                long_beacon_area[9] += 1
                            elif reach_half_y and ((reach_quarter_x and not reach_quarter34_x) or (not reach_quarter_x and reach_quarter34_x)) and not reach_half_x and not reach_edge_x and (highest_y-lowest_y)/height_half_point > 1.0:
                                mid_contour_count += 1
                                long_beacon_area[10] += 1
                            elif ((reach_quarter_x_half_y and not reach_quarter34_x) or (reach_quarter34_x_half_y and not reach_quarter_x)) and reach_middle_third_count < 0 and reach_near_edge_x_count > -30 and (highest_y-lowest_y)/height_half_point > 1.0:
                                mid_contour_count += 1
                                long_beacon_area[11] += 1
                            elif reach_half_y and not reach_edge_x and not reach_half_x and not reach_edge_y_bottom and (highest_y-lowest_y)/height_half_point >= 0.7:
                                mid_contour_count += 1
                                long_beacon_area[12] += 1
                        elif classes[bi] == 0.:
                            if reach_half_x and reach_half_y and not (reach_edge_x or reach_edge_y) and (highest_y-lowest_y)/height_half_point > 0.35 and (highest_x-lowest_x) > (highest_y-lowest_y):
                                mid_contour_count += 1
                        elif classes[bi] == 2.:
                            if reach_half_x and reach_half_y and not (reach_edge_x or reach_edge_y) and (highest_y-lowest_y)/height_half_point > 0.54:
                                mid_contour_count += 1
                        elif classes[bi] == 3.:
                            if reach_half_x and not (reach_edge_x or reach_edge_y):
                                mid_contour_count += 1
                            elif reach_quarter_x and reach_half_y and not (reach_edge_x or reach_edge_y):
                                mid_contour_count += 1
                        else:
                            if reach_half_x and reach_half_y and not (reach_edge_x or reach_edge_y):
                                mid_contour_count += 1

                if mid_contour_count > 0:
                    found_active = False
                    active_beacon_count += 1
                    if active_beacon_count >= 4 and frame_count <= 100:
                        not_long_beacon_active = True
                        if sum(long_beacon_area) >= 4:
                            count_double_type = 0
                            count_multiple_type = 0
                            for a in long_beacon_area:
                                if a >= 2:
                                    count_double_type += 1
                                if a >= 1:
                                    count_multiple_type += 1
                                if count_double_type > 1 or count_multiple_type >= 3:
                                    found_active = True
                                    long_beacon_area = [0,0,0,0,0,0,0,0,0,0,0,0,0]
                                    not_long_beacon_active = False
                                    break 
                            if not_long_beacon_active and (long_beacon_area[1] >= 15 or long_beacon_area[6] >= 4 or long_beacon_area[9] >= 4 or long_beacon_area[4] >= 8):
                                found_active = True
                                long_beacon_area = [0,0,0,0,0,0,0,0,0,0,0,0,0]
                                not_long_beacon_active = False
                                frame_count = 0
                                active_beacon_count = 0
                                beacon_type_count = [0,0,0,0,0]
                            elif not_long_beacon_active and (long_beacon_area[3] >= 8 or long_beacon_area[10] >= 6 or long_beacon_area[11] >= 6):
                                found_active = True
                                long_beacon_area = [0,0,0,0,0,0,0,0,0,0,0,0,0]
                                not_long_beacon_active = False
                                frame_count = 0
                                active_beacon_count = 0
                                beacon_type_count = [0,0,0,0,0]
                        if not_long_beacon_active and [i for i in beacon_type_count[:-1] if i > 4] and not [i for i in long_beacon_area if i >= 4]:
                            found_active = True
                            active_beacon_count = 0
                            frame_count = 0
                            beacon_type_count = [0,0,0,0,0]
                    if frame_count >= 100:
                        frame_count = 0
                        active_beacon_count = 0
                        beacon_type_count = [0,0,0,0,0]
                        long_beacon_area = [0,0,0,0,0,0,0,0,0,0,0,0,0]
                    if found_active:
                        print("Found an active emergency vehicle, sending message")
                        sock2.sendto(b'found', (UDP_IP, UDP_PORT))
            else:
                print("No detection")
        else:
            print("No detection")
        # just for displaying the current detection using images
        desired_width = 960
        desired_height = 960

        if img.width > img.height:
            if img.width > desired_width:
                multiplier = round(img.width/desired_width)
                width = img.width/multiplier
                height = img.height/multiplier
            else:
                multiplier = round(desired_width/img.width)
                width = img.width*multiplier
                height = img.height*multiplier
        else:
            if img.height > desired_height:
                multiplier = round(img.height/desired_height)
                width = img.width/multiplier
                height = img.height/multiplier
            else:
                multiplier = round(desired_height/img.height)
                width = img.width*multiplier
                height = img.height*multiplier

        imgcv = cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR)
        resize_img = cv2.resize(imgcv, dsize=(int(width), int(height)), interpolation=cv2.INTER_CUBIC)
        cv2.imshow('image', resize_img)
        cv2.waitKey(10) 
             
        frame_count += 1
        currentframe += 1
    except Exception as e:
        print(e)
        sock.close()
        print("CLose")
cv2.destroyAllWindows()